<a href="https://colab.research.google.com/github/anjali-ojha/deep-learning/blob/a10/assignment/assignment-10/Anjali_Ojha_HW_10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# BERT for QnA:

# 1. Load the SQUAD 2.0 (https://pytorch.org/text/stable/datasets.html#torchtext.datasets.SQuAD2) dataset from pytorch dataset library. take minimum 20 QnA pairs.




In [1]:
! pip install datasets
! pip install portalocker
! pip install transformers datasets torch
! pip install transformers[torch]
! pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 19.9 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3

In [2]:
from datasets import load_dataset

# Load the SQuAD 2.0 dataset
squad_dataset = load_dataset("squad_v2")

# Get the training data
train_data = squad_dataset["train"]

# Get the development data
dev_data = squad_dataset["validation"]
train_data

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/130319 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11873 [00:00<?, ? examples/s]

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 130319
})

In [3]:
train_data.features

{'id': Value(dtype='string', id=None),
 'title': Value(dtype='string', id=None),
 'context': Value(dtype='string', id=None),
 'question': Value(dtype='string', id=None),
 'answers': Sequence(feature={'text': Value(dtype='string', id=None), 'answer_start': Value(dtype='int32', id=None)}, length=-1, id=None)}

In [4]:
train_data['answers'][:2]

[{'text': ['in the late 1990s'], 'answer_start': [269]},
 {'text': ['singing and dancing'], 'answer_start': [207]}]

# 2. Display a few raw QnA data samples.


In [5]:
for i in range(5):
  print(f"\nsample = {i+1}, \n\t Ques = {train_data['question'][i]}, \n\t Ans  = {train_data['answers'][i]['text'][0]}")
# train_data['question'][:2]


sample = 1, 
	 Ques = When did Beyonce start becoming popular?, 
	 Ans  = in the late 1990s

sample = 2, 
	 Ques = What areas did Beyonce compete in when she was growing up?, 
	 Ans  = singing and dancing

sample = 3, 
	 Ques = When did Beyonce leave Destiny's Child and become a solo singer?, 
	 Ans  = 2003

sample = 4, 
	 Ques = In what city and state did Beyonce  grow up? , 
	 Ans  = Houston, Texas

sample = 5, 
	 Ques = In which decade did Beyonce become famous?, 
	 Ans  = late 1990s


# 3. Preprocess/Clean dataset as per BERT format.

In [ ]:
from transformers import BertTokenizer, BertForQuestionAnswering

# Initialize tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForQuestionAnswering.from_pretrained('bert-base-uncased')

N = 100

def preprocess_qa(example):
    question, context, answer = example['question'], example['context'], example['answers']['text'][0]
    inputs = tokenizer(question, context, padding='max_length', truncation=True, max_length=512, return_tensors='pt')
    inputs.update({'labels': tokenizer(answer, padding='max_length', truncation=True, max_length=32, return_tensors='pt')['input_ids']})
    return inputs


# Define your filter function
def filter_function(example, index = -1):
    # Example of a simple filter, you can replace this with your own logic
    return i <= N and index >len(example['answers']['text']) > 0

# Filter the train data
filtered_train_data = train_data.filter(filter_function,  with_indices=True)

train_data = filtered_train_data.map(preprocess_qa)



Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Filter:   0%|          | 0/130319 [00:00<?, ? examples/s]

Map:   0%|          | 0/86819 [00:00<?, ? examples/s]

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

In [ ]:
# Filter the train data
filtered_dev_data = dev_data.filter(filter_function, with_indices=True))

# train_data = filtered_train_data.map(preprocess_qa)

dev_data = filtered_dev_data.map(preprocess_qa)


# 4. Train the BERT QnA model. Evaluate the model.


In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    per_device_train_batch_size=4,
    num_train_epochs=2,
    logging_dir='./logs',
    output_dir = "./output",
    logging_steps=1000,
    save_steps=1000,
    evaluation_strategy='steps',
    eval_steps=1000,
    load_best_model_at_end=True,
    metric_for_best_model='exact_match',
    save_total_limit=2,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=dev_data,
    tokenizer=tokenizer,
)

trainer.train()

# 5. Perform an Inference and show the predicted vs ground truth answers.

In [ ]:
results = trainer.evaluate()
print(results)

In [ ]:
! pip install transformers[torch]